In [1]:
import sqlite3, shutil, json

In [2]:
shutil.copy("openstudio_refrigeration_system.db", "openstudio_refrigeration_system_wk.db")
db_path="openstudio_refrigeration_system_wk.db"

##### Select Cases and walk-in units

In [175]:
def get_data_from_db(db_path, selected_case_units, selected_walkin_units):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    case_data = {}
    walkin_data = {}

    # CASES
    case_counts = {}
    for unit in selected_case_units:
        key = unit.case_name.lower()
        case_counts[key] = case_counts.get(key, 0) + unit.number_of_units

    for case_name, count in case_counts.items():
        cursor.execute("""
            SELECT 
                case_name, template, operation_type,
                rated_capacity, unit_length, case_operating_temperature,
                evaporator_temperature, fan_power, lighting_power,
                defrost_type, defrost_schedules, drip_down_schedules,
                case_lighting_schedules, fraction_of_lighting_energy_to_case,
                anti_sweat_power, anti_sweat_heater_control_type,
                fraction_of_anti_sweat_heater_energy_to_cases,
                rated_latent_heat_ratio, rated_runtime_fraction,
                latent_case_credit_curve_type, latent_case_credit_curve_name,
                defrost_energy_correction_curve_type, defrost_energy_correction_curve_name,
                HVAC_return_air_fraction, restocking_schedule, case_credit_fraction_schedule
            FROM refrigeration_cases
            WHERE lower(case_name) = ?
        """, (case_name,))
        row = cursor.fetchone()

        if row:
            name = row[0]
            case_data[name] = {
                "template": row[1],
                "operation_type": row[2],
                "rated_capacity": row[3],
                "unit_length": row[4],
                "unit_count": count,
                "total_rated_capacity": row[3] * row[4] * count,
                "case_operating_temperature": row[5],
                "evaporator_temperature": row[6],
                "fan_power_per_unit_length": row[7],
                "lighting_power_per_unit_length": row[8],
                "defrost_type": row[9],
                "defrost_schedule": row[10],
                "drip_down_schedule": row[11],
                "case_lighting_schedule": row[12],
                "fraction_of_lighting_energy_to_case": row[13],
                "anti_sweat_power": row[14],
                "anti_sweat_heater_control_type": row[15],
                "fraction_of_anti_sweat_heater_energy_to_cases": row[16],
                "rated_latent_heat_ratio": row[17],
                "rated_runtime_fraction": row[18],
                "latent_case_credit_curve_type": row[19],
                "latent_case_credit_curve_name": row[20],
                "defrost_energy_correction_curve_type": row[21],
                "defrost_energy_correction_curve_name": row[22],
                "HVAC_return_air_fraction": row[23],
                "restocking_schedule": row[24],
                "case_credit_fraction_schedule": row[25],
            }

    # WALKINS
    walkin_counts = {}
    for unit in selected_walkin_units:
        key = unit.walkin_name.lower()
        walkin_counts[key] = walkin_counts.get(key, 0) + unit.number_of_units

    for walkin_name, count in walkin_counts.items():
        cursor.execute("""
            SELECT 
                walkin_name, template, operation_type,
                rated_capacity, operating_temperature,
                rated_cooling_fan_power, lighting_power, lighting_schedule,
                defrost_type, defrost_control_type, defrost_schedule, drip_down_schedule,
                stocking_door_u, area_of_stocking_doors_facing_zone, stocking_door_schedule,
                reachin_door_uvalue, area_of_glass_reachin_doors_facing_zone
            FROM refrigeration_walkins
            WHERE lower(walkin_name) = ?
        """, (walkin_name,))
        row = cursor.fetchone()
        if row:
            walkin_data[row[0]] = {
                "template": row[1],
                "operation_type": row[2],
                "rated_capacity": row[3],
                "number_of_units": count,
                "total_rated_capacity": row[3] * count,
                "operating_temperature": row[4],
                "rated_cooling_fan_power": row[5],
                "lighting_power": row[6],
                "lighting_schedule": row[7],
                "defrost_type": row[8],
                "defrost_control_type": row[9],
                "defrost_schedule": row[10],
                "drip_down_schedule": row[11],
                "stocking_door_u": row[12],
                "area_of_stocking_doors_facing_zone": row[13],
                "stocking_door_schedule": row[14],
                "reachin_door_uvalue": row[15],
                "area_of_glass_reachin_doors_facing_zone": row[16],
            }

    conn.close()
    return case_data, walkin_data

#### SuperMarket Default setting

In [176]:
class BuildingUnit:
    def __init__(self, building_type, base_name, category, number_of_units=None, template=None, user_mode=False, zone_name=None):
        self.building_type = building_type
        self.base_name = base_name
        self.category = category
        self.number_of_units = number_of_units if number_of_units is not None else 1
        self.template = template
        self.user_mode = user_mode
        self.zone_name = zone_name or ("MainSales" if "walk-in" not in base_name.lower() else "ActiveStorage")
        
        # user model : 
        if self.user_mode:
            self.case_name = base_name
            self.walkin_name = base_name
            self.osm_name = f"User {template} {base_name}"
        # automated modeL: osm name- building_type, template, base_name 
        else:
            self.case_name = f"{self.template} {self.base_name.replace(',', '')}"
            self.walkin_name = f"{self.template} {self.base_name.replace(',', '')}"
            self.osm_name = f"{building_type} {template} {base_name} - {category}"
            
        # case_name: template+base name 
        self.case_name = f"{self.template} {self.base_name.replace(',', '')}"

        # walkin_name: template+base name 
        self.walkin_name = f"{self.template} {self.base_name.replace(',', '')}"

    def __repr__(self):
        if self.number_of_units is not None:
            return f"\"osm name\": \"{self.osm_name}\", \"case_name\": \"{self.case_name}\", \"number_of_units\": {self.number_of_units}"
        else:
            return f"\"osm name\": \"{self.osm_name}\", \"walkin_name\": \"{self.walkin_name}\", \"number_of_units\": {self.number_of_units}"
    
class SuperMarketSystem:
    def __init__(self, system_type, db_path):
        self.building_type = "SuperMarket"
        self.db_path = db_path
        self.system_type = system_type

    def load_defaults(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # CASE INFO (includes number_of_units)
        cursor.execute("""
            SELECT base_name, category, number_of_units
            FROM building_category_mapping
            WHERE building_type = 'SuperMarket' 
            AND system_type = 'case'
            AND template = ?;
        """, (self.system_type,))
        case_results = cursor.fetchall()
        self.cases = [BuildingUnit(self.building_type, base, category, qty, self.system_type) for base, category, qty in case_results]

        # WALK-IN INFO
        cursor.execute("""
            SELECT base_name, category
            FROM building_category_mapping
            WHERE building_type = 'SuperMarket' 
            AND system_type = 'walkin'
            AND template = ?;
        """, (self.system_type,))
        walkin_results = cursor.fetchall()
        self.walkins = [BuildingUnit(self.building_type, base, category, template=self.system_type) for base, category in walkin_results]

        conn.close()

# template selection
def get_valid_template():
    valid_templates = ["old", "new", "advanced"]
    template = input("Choose template (old/new/advanced): ").lower()
    while template not in valid_templates:
        print("Invalid template. Please choose from 'old', 'new', or 'advanced'.")
        template = input("Choose template (old/new/advanced): ").lower()
    return template

#### test mode selection: user input selection & automated input selection

In [177]:
# test mode selection
def select_test_mode():
    mode = input("Select test mode (user/automated): ").lower()
    while mode not in ["user", "automated"]:
        print("Invalid mode. Please choose from 'user' or 'automated'.")
        mode = input("Select test mode (user/automated): ").lower()
    return mode

# User mode selection
def user_mode():
    selected_case_units = []
    selected_walkin_units = []
    selected_template = get_valid_template()
    
    print("\n--- Add Case Units ---")
    while True:
        case_name = input("Enter case name (or 'done' to finish): ")
        if case_name.lower() == 'done':
            break
        number_of_units = int(input(f"Enter number of units for {case_name}: "))
        selected_case_units.append(
            BuildingUnit("User", case_name, "Category", number_of_units, template=template, user_mode=True, zone_name=zone_name)
        )    
        
    print("\n--- Add Walk-in Units ---")
    while True:
        walkin_name = input("Enter walk-in name (or 'done' to finish): ")
        if walkin_name.lower() == 'done':
            break

        selected_walkin_units.append(
            BuildingUnit("User", walkin_name, "Category", 1, template=template, user_mode=True, zone_name=zone_name)
        )
    return selected_case_units, selected_walkin_units, selected_template


# Automated mode selection
def automated_mode():
    building_types = ["SuperMarket", "TBD"]  # future addition

    # building_type selection
    print("Choose building type:")
    for idx, building_type in enumerate(building_types, 1):
        print(f"{idx}. {building_type}")
    
    choice = int(input("Enter the number of your choice: "))
    while choice < 1 or choice > len(building_types):
        print("Invalid choice. Please try again.")
        choice = int(input("Enter the number of your choice: "))
    
    selected_building_type = building_types[choice - 1]

    print(f"Chosen building type: {selected_building_type}")
    
    # template selection
    selected_template = get_valid_template()

    # system load
    system = None
    if selected_building_type == "SuperMarket":
        system = SuperMarketSystem(selected_template, db_path)
    elif selected_building_type == "TBD":
        system = TBD(selected_template, db_path)  # future addition
    
    if system:
        system.load_defaults()
        selected_case_units = system.cases
        selected_walkin_units = system.walkins

    return selected_case_units, selected_walkin_units, selected_template

def get_building_name():
    global_vars = globals()
    if "mode" in global_vars:
        if global_vars["mode"] == "automated":
            return global_vars.get("selected_building_type", "Automated Example")
        elif global_vars["mode"] == "user":
            return "User Defined System"
    return "OpenStudio Refrigeration System"

#### automated mode test

In [178]:
mode = select_test_mode()

if mode == "user":
    selected_case_units, selected_walkin_units, selected_template = user_mode()
elif mode == "automated":
    selected_case_units, selected_walkin_units, selected_template = automated_mode()

# Selected Case Units 
print("\nSelected Case Units:")
for unit in selected_case_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"case_name\": \"{unit.case_name}\", \"number_of_units\": {unit.number_of_units}")

# Selected Walk-in Units 
print("\nSelected Walk-in Units:")
for unit in selected_walkin_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"walkin_name\": \"{unit.walkin_name}\", \"number_of_units\": {unit.number_of_units}")
  

Select test mode (user/automated): automated
Choose building type:
1. SuperMarket
2. TBD
Enter the number of your choice: 1
Chosen building type: SuperMarket
Choose template (old/new/advanced): advanced

Selected Case Units:
"osm name": "SuperMarket advanced LT Coffin - Ice Cream - Ice Cream", "case_name": "advanced LT Coffin - Ice Cream", "number_of_units": 1.0
"osm name": "SuperMarket advanced LT Reach-In - Ice Cream - Ice Cream", "case_name": "advanced LT Reach-In - Ice Cream", "number_of_units": 14.0
"osm name": "SuperMarket advanced LT Reach-In - Frozen Food - Frozen Food", "case_name": "advanced LT Reach-In - Frozen Food", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Deli", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Produce", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 10.0
"osm name": "SuperMarket advance

#### Allocate cases to each rack

In [179]:
def assign_racks_to_cases_and_walkins(db_path, selected_case_units, selected_walkin_units, default_max_capacity=30000):
    # get case and walkin data from DB
    case_data, walkin_data = get_data_from_db(db_path, selected_case_units, selected_walkin_units)

    # Determine template from selected units
    template = None
    if selected_case_units:
        template = selected_case_units[0].template.lower()
    elif selected_walkin_units:
        template = selected_walkin_units[0].template.lower()

    # Set MT and LT limits based on template
    if template == "advanced":
        max_mt_capacity = 30000
        max_lt_capacity = 15000
    elif template in ["old", "new"]:
        max_mt_capacity = 50000
        max_lt_capacity = 25000
    else:
        max_mt_capacity = max_lt_capacity = default_max_capacity  # fallback    

    # assign cases and walkins to MT rack and LT rack
    mt_racks = []
    lt_racks = []

    def distribute_units(data, racks, max_capacity_per_rack, is_walkin=False):
        units = sorted(
            data.items(),
            key=lambda x: x[1].get('total_rated_capacity', x[1].get('rated_capacity', 0)),
            reverse=True
        )

        current_rack = []
        current_capacity = 0
        rack_index = 1

        for name, item in units:
            total_capacity = item.get('total_rated_capacity') or item.get('rated_capacity')

            if current_capacity + total_capacity <= max_capacity_per_rack:
                current_rack.append({'name': name, 'capacity': total_capacity})
                current_capacity += total_capacity
            else:
                if current_rack:
                    for unit in current_rack:
                        if unit['name'] in data:
                            data[unit['name']]['assigned_rack'] = rack_index
                    racks.append(current_rack)
                    rack_index += 1
                current_rack = [{'name': name, 'capacity': total_capacity}]
                current_capacity = total_capacity

        if current_rack:
            for unit in current_rack:
                if unit['name'] in data:
                    data[unit['name']]['assigned_rack'] = rack_index
            racks.append(current_rack)

    mt_case_data = {name: item for name, item in case_data.items() if item.get('operation_type') == 'MT'}
    mt_walkin_data = {name: item for name, item in walkin_data.items() if item.get('operation_type') == 'MT'}
    lt_case_data = {name: item for name, item in case_data.items() if item.get('operation_type') == 'LT'}
    lt_walkin_data = {name: item for name, item in walkin_data.items() if item.get('operation_type') == 'LT'}

    # combine MT/LT data
    mt_combined = {**mt_case_data, **mt_walkin_data}
    lt_combined = {**lt_case_data, **lt_walkin_data}

    # Assign to MT/LT racks and embed rack numbers
    distribute_units(mt_combined, mt_racks, max_mt_capacity)
    distribute_units(lt_combined, lt_racks, max_lt_capacity)

    # Update case_data and walkin_data with assigned rack info
    case_data.update(mt_case_data)
    case_data.update(lt_case_data)
    walkin_data.update(mt_walkin_data)
    walkin_data.update(lt_walkin_data)

    return mt_racks, lt_racks, case_data, walkin_data

def display_rack_capacity(racks, selected_units, rack_type=""):
    print(f"\n{rack_type} Racks:")
    name_to_osm = {}
    for unit in selected_units:
        name_to_osm[unit.case_name] = unit.osm_name
        name_to_osm[unit.walkin_name] = unit.osm_name  # walkin도 포함

    for i, rack in enumerate(racks, 1):
        total_capacity = sum(item['capacity'] for item in rack)
        print(f"Rack {i}: Total Capacity = {total_capacity:.2f} W")
        for item in rack:
            original_name = item['name']
            osm_display = name_to_osm.get(original_name, original_name)  # fallback 처리
            print(f"  - {osm_display} : {item['capacity']:.2f} W")
        print()

#### rack assigment TEST

In [180]:
mt_racks, lt_racks, case_data, walkin_data = assign_racks_to_cases_and_walkins(
    db_path, selected_case_units, selected_walkin_units
)

# Display rack capacity
display_rack_capacity(mt_racks, selected_case_units + selected_walkin_units, rack_type="MT")
display_rack_capacity(lt_racks, selected_case_units + selected_walkin_units, rack_type="LT")


MT Racks:
Rack 1: Total Capacity = 81644.64 W
  - SuperMarket advanced MT Vertical Open - Others - Prepared Foods : 81644.64 W

Rack 2: Total Capacity = 25132.32 W
  - SuperMarket advanced MT Vertical Open - Meat - Meat : 25132.32 W

Rack 3: Total Capacity = 29896.18 W
  - SuperMarket advanced MT Service - Others - Bakery : 10385.28 W
  - SuperMarket advanced MT Vertical Open - Beverage - Beverage : 7039.20 W
  - SuperMarket advanced MT Walk-in Cooler - 660SF with glass door - Dairy  : 6460.70 W
  - SuperMarket advanced MT Walk-in Cooler - 600SF with no glass door - Meat Prep : 6011.00 W

Rack 4: Total Capacity = 25574.20 W
  - SuperMarket advanced MT Service - Meat - Meat : 5504.40 W
  - SuperMarket advanced MT Walk-in Cooler - 480SF with no glass door - Meat : 5085.30 W
  - SuperMarket advanced MT Walk-in Cooler - 480SF with glass door - Beer : 5085.30 W
  - SuperMarket advanced MT Walk-in Cooler - 400SF with no glass door - Produce  : 4448.80 W
  - SuperMarket advanced MT Walk-in C

#### user selection mode test

In [156]:
mode = select_test_mode()

if mode == "user":
    selected_case_units, selected_walkin_units, selected_template = user_mode()
    building_name = "User Defined Example"
elif mode == "automated":
    selected_case_units, selected_walkin_units, selected_template = automated_mode()
    building_name = selected_case_units[0].building_type  # e.g., "SuperMarket"

# Selected Case Units 
print("\n 🛒 Selected Case Units:")
for unit in selected_case_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"case_name\": \"{unit.case_name}\", \"number_of_units\": {unit.number_of_units}")

# Selected Walk-in Units 
print("\n 📦 Selected Walk-in Units:")
for unit in selected_walkin_units:
    print(f"\"osm name\": \"{unit.osm_name}\", \"walkin_name\": \"{unit.walkin_name}\", \"number_of_units\": {unit.number_of_units}")
  

Select test mode (user/automated): automated
Choose building type:
1. SuperMarket
2. TBD
Enter the number of your choice: 1
Chosen building type: SuperMarket
Choose template (old/new/advanced): advanced

 🛒 Selected Case Units:
"osm name": "SuperMarket advanced LT Coffin - Ice Cream - Ice Cream", "case_name": "advanced LT Coffin - Ice Cream", "number_of_units": 1.0
"osm name": "SuperMarket advanced LT Reach-In - Ice Cream - Ice Cream", "case_name": "advanced LT Reach-In - Ice Cream", "number_of_units": 14.0
"osm name": "SuperMarket advanced LT Reach-In - Frozen Food - Frozen Food", "case_name": "advanced LT Reach-In - Frozen Food", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Deli", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 14.0
"osm name": "SuperMarket advanced MT Island - Deli Produce Case - Produce", "case_name": "advanced MT Island - Deli Produce Case", "number_of_units": 10.0
"osm name": "SuperMarket adva

In [158]:
mtmt_racks, lt_racks, case_data, walkin_data = assign_racks_to_cases_and_walkins(
    db_path, selected_case_units, selected_walkin_units
)

# Display rack capacity
display_rack_capacity(mt_racks, selected_case_units + selected_walkin_units, rack_type="MT")
display_rack_capacity(lt_racks, selected_case_units + selected_walkin_units, rack_type="LT")


MT Racks:
Rack 1: Total Capacity = 81644.64 W
  - SuperMarket advanced MT Vertical Open - Others - Prepared Foods : 81644.64 W

Rack 2: Total Capacity = 25132.32 W
  - SuperMarket advanced MT Vertical Open - Meat - Meat : 25132.32 W

Rack 3: Total Capacity = 29896.18 W
  - SuperMarket advanced MT Service - Others - Bakery : 10385.28 W
  - SuperMarket advanced MT Vertical Open - Beverage - Beverage : 7039.20 W
  - SuperMarket advanced MT Walk-in Cooler - 660SF with glass door - Dairy  : 6460.70 W
  - SuperMarket advanced MT Walk-in Cooler - 600SF with no glass door - Meat Prep : 6011.00 W

Rack 4: Total Capacity = 25574.20 W
  - SuperMarket advanced MT Service - Meat - Meat : 5504.40 W
  - SuperMarket advanced MT Walk-in Cooler - 480SF with no glass door - Meat : 5085.30 W
  - SuperMarket advanced MT Walk-in Cooler - 480SF with glass door - Beer : 5085.30 W
  - SuperMarket advanced MT Walk-in Cooler - 400SF with no glass door - Produce  : 4448.80 W
  - SuperMarket advanced MT Walk-in C

#### generate case and walkin objects

In [181]:
def generate_case_objects_from_data(case_data, selected_case_units):
    """Generate OS:Refrigeration:Case JSON objects based on database data and unit zones."""
    name_to_osm = {unit.case_name: unit.osm_name for unit in selected_case_units}
    name_to_zone = {unit.case_name: unit.zone_name for unit in selected_case_units}

    objects = []
    for case_name, info in case_data.items():
        osm_name = name_to_osm.get(case_name, case_name)
        zone_name = name_to_zone.get(case_name, "MainSales")

        obj = {
            "type": "OS:Refrigeration:Case",
            "name": osm_name,
            "ZoneName": zone_name,
            "CaseLength": info.get("unit_length"),
            "RatedTotalCoolingCapacity": info.get("rated_capacity"),
            "OperatingTemperature": info.get("case_operating_temperature"),
            "EvaporatorTemperature": info.get("evaporator_temperature"),
            "FanPowerPerUnitLength": info.get("fan_power_per_unit_length"),
            "LightingPowerPerUnitLength": info.get("lighting_power_per_unit_length"),
            "DefrostType": info.get("defrost_type"),
            "DefrostSchedule": info.get("defrost_schedule"),
            "DripDownSchedule": info.get("drip_down_schedule"),
            "CaseLightingScheduleName": info.get("case_lighting_schedule")
        }
        objects.append(obj)
    return objects


def generate_walkin_objects_from_data(walkin_data, selected_walkin_units):
    """Generate OS:Refrigeration:WalkIn JSON objects based on database data and unit zones."""
    name_to_osm = {unit.walkin_name: unit.osm_name for unit in selected_walkin_units}
    name_to_zone = {unit.walkin_name: unit.zone_name for unit in selected_walkin_units}

    objects = []
    for walkin_name, info in walkin_data.items():
        osm_name = name_to_osm.get(walkin_name, walkin_name)
        zone_name = name_to_zone.get(walkin_name, "ActiveStorage")

        obj = {
            "type": "OS:Refrigeration:WalkIn",
            "name": osm_name,
            "ZoneName": zone_name,
            "RatedCoolingCapacity": info.get("rated_capacity"),
            "OperatingTemperature": info.get("operating_temperature"),
            "CoolingFanPower": info.get("rated_cooling_fan_power"),
            "LightingPower": info.get("lighting_power"),
            "LightingScheduleName": info.get("lighting_schedule"),
            "DefrostType": info.get("defrost_type"),
            "DefrostControlType": info.get("defrost_control_type"),
            "DefrostScheduleName": info.get("defrost_schedule"),
            "DripDownScheduleName": info.get("drip_down_schedule"),
            "StockingDoorUValue": info.get("stocking_door_u"),
            "StockingDoorAreaFacingZone": info.get("area_of_stocking_doors_facing_zone"),
            "StockingDoorScheduleName": info.get("stocking_door_schedule"),
            "GlassReachInDoorUValue": info.get("reachin_door_uvalue"),
            "GlassReachInDoorAreaFacingZone": info.get("area_of_glass_reachin_doors_facing_zone")
        }
        objects.append(obj)
    return objects

In [182]:
case_objects = generate_case_objects_from_data(case_data, selected_case_units)
walkin_objects = generate_walkin_objects_from_data(walkin_data, selected_walkin_units)

#### expoert to JSON files

In [183]:
def export_cases_and_walkins_to_json(
    case_objects,
    walkin_objects,
    output_path="Cases_and_Walkins.json",
    building_name="Supermarket Refrigeration System"
):
    """
    Export refrigeration case and walk-in objects to an OpenStudio JSON file including fixed zone objects.

    Args:
        case_objects (list): List of OS:Refrigeration:Case JSON objects.
        walkin_objects (list): List of OS:Refrigeration:WalkIn JSON objects.
        output_path (str): Path to save the JSON file.
        building_name (str): Label for the JSON file.
    """
    # Fixed zone names
    zone_objects = [
        {"type": "OS:ThermalZone", "name": "MainSales"},
        {"type": "OS:ThermalZone", "name": "ActiveStorage"}
    ]

    all_objects = zone_objects + case_objects + walkin_objects

    openstudio_json = {
        "Version": "0.2.1",
        "Building": building_name,
        "objects": all_objects
    }

    with open(output_path, "w") as f:
        json.dump(openstudio_json, f, indent=4)

    print(f"✅ Refrigeration case + walk-in JSON (with zones) saved to: {output_path}")
    print("\n📦 JSON Preview:")
    print(json.dumps(openstudio_json, indent=2))

In [237]:
export_cases_and_walkins_to_json(case_objects, walkin_objects, output_path="Refrigeration_System_List.json")

✅ Refrigeration case + walk-in JSON (with zones) saved to: Refrigeration_System_List.json

📦 JSON Preview:
{
  "Version": "0.2.1",
  "Building": "Supermarket Refrigeration System",
  "objects": [
    {
      "type": "OS:ThermalZone",
      "name": "MainSales"
    },
    {
      "type": "OS:ThermalZone",
      "name": "ActiveStorage"
    },
    {
      "type": "OS:Refrigeration:Case",
      "name": "SuperMarket advanced LT Coffin - Ice Cream - Ice Cream",
      "ZoneName": "MainSales",
      "CaseLength": 2.0,
      "RatedTotalCoolingCapacity": 331.8,
      "OperatingTemperature": -27.2,
      "EvaporatorTemperature": -31.1,
      "FanPowerPerUnitLength": 8.75,
      "LightingPowerPerUnitLength": 16.4,
      "DefrostType": "ElectricwithTemperatureTerminatiOn",
      "DefrostSchedule": "LT Coffin Defrost Sch",
      "DripDownSchedule": "LT Coffin Drip-Down Sch",
      "CaseLightingScheduleName": "Always On"
    },
    {
      "type": "OS:Refrigeration:Case",
      "name": "advanced LT Re

#### generate compressor specs

In [185]:
def calculate_compressors_for_racks(racks, rack_type, template, redundancy=True):
    capacity, _, _, _ = get_compressor_specs(template, rack_type)

    compressors_per_rack = []
    for i, rack in enumerate(racks, 1):
        total_capacity = sum(item['capacity'] for item in rack)
        compressors = total_capacity / capacity
        compressors = int(compressors) + (1 if compressors % 1 > 0 else 0)

        if redundancy:
            compressors += 1

        compressors_per_rack.append({
            "rack_number": i,
            "rack_load": total_capacity,
            "compressors_needed": compressors
        })

    return compressors_per_rack

def get_compressor_specs(template, operation_type):
    """Return compressor specs: capacity (W), power (W), COP, EER."""
    if operation_type == "MT":
        if template == "old":
            return 52733.94, 24945, 2.12, 7.22
        else:
            return 38099.93, 15448, 2.47, 8.42
    elif operation_type == "LT":
        if template == "old":
            return 20038.77, 13963, 1.44, 4.90
        else:
            return 17181.96, 9766, 1.76, 6.00
    else:
        raise ValueError(f"Unknown operation type: {operation_type}")

#### summarize MT/LT compressor assignment

In [186]:
def summarize_compressor_assignment(mt_racks, lt_racks, selected_template):
    """
    Calculate and display compressor assignment and specs based on rack loads and template.
    
    Args:
        mt_racks (list): List of MT rack load info
        lt_racks (list): List of LT rack load info
        selected_template (str): Template type ('old', 'new', 'advanced')
    
    Returns:
        Tuple: (mt_info, lt_info) list of compressor assignments for each rack
    """
    mt_info = calculate_compressors_for_racks(mt_racks, "MT", template=selected_template)
    lt_info = calculate_compressors_for_racks(lt_racks, "LT", template=selected_template)

    print("\n🧊 MT Rack Compressor Assignment:")
    for info in mt_info:
        print(f"Rack {info['rack_number']}: Load = {info['rack_load']:.2f} W → Compressors Needed = {info['compressors_needed']}")

    print("\n❄️ LT Rack Compressor Assignment:")
    for info in lt_info:
        print(f"Rack {info['rack_number']}: Load = {info['rack_load']:.2f} W → Compressors Needed = {info['compressors_needed']}")

    # Specs 출력
    mt_capacity, mt_power, mt_cop, mt_eer = get_compressor_specs(selected_template, "MT")
    lt_capacity, lt_power, lt_cop, lt_eer = get_compressor_specs(selected_template, "LT")

    print(f"\n⚙️ Compressor Specs for the selected template '{selected_template}':")
    print(f"🧊 MT → Capacity: {mt_capacity:.2f} W, Power: {mt_power:.2f} W, COP: {mt_cop:.2f}, EER: {mt_eer:.2f}")
    print(f"❄️ LT → Capacity: {lt_capacity:.2f} W, Power: {lt_power:.2f} W, COP: {lt_cop:.2f}, EER: {lt_eer:.2f}")

    return mt_info, lt_info

In [187]:
mt_info, lt_info = summarize_compressor_assignment(mt_racks, lt_racks, selected_template) 


🧊 MT Rack Compressor Assignment:
Rack 1: Load = 81644.64 W → Compressors Needed = 4
Rack 2: Load = 25132.32 W → Compressors Needed = 2
Rack 3: Load = 29896.18 W → Compressors Needed = 2
Rack 4: Load = 25574.20 W → Compressors Needed = 2

❄️ LT Rack Compressor Assignment:
Rack 1: Load = 17623.20 W → Compressors Needed = 3
Rack 2: Load = 16753.80 W → Compressors Needed = 2
Rack 3: Load = 663.60 W → Compressors Needed = 2

⚙️ Compressor Specs for the selected template 'advanced':
🧊 MT → Capacity: 38099.93 W, Power: 15448.00 W, COP: 2.47, EER: 8.42
❄️ LT → Capacity: 17181.96 W, Power: 9766.00 W, COP: 1.76, EER: 6.00


#### read compressor curves

In [188]:
def get_compressor_curve(db_path, template, operation_type, curve_type=None):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    query = """
    SELECT curve_name, coefficient1, coefficient2, coefficient3, coefficient4, 
           coefficient5, coefficient6, coefficient7, coefficient8, coefficient9, coefficient10,
           min_val_x, max_val_x, min_val_y, max_val_y
    FROM refrigeration_compressors
    WHERE template = ? AND operation_type = ?
    """
    params = [template, operation_type]

    if curve_type:
        query += " AND curve_type = ?"
        params.append(curve_type)

    cursor.execute(query, params)
    row = cursor.fetchone()
    conn.close()

    if not row:
        return None

    (curve_name, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10,
     min_x, max_x, min_y, max_y) = row

    curve_json = {
        "type": "OS:Curve:Bicubic",
        "name": curve_name,
        "Coefficient1Constant": c1,
        "Coefficient2x": c2,
        "Coefficient3x2": c3,
        "Coefficient4y": c4,
        "Coefficient5y2": c5,
        "Coefficient6xy": c6,
        "Coefficient7x3": c7,
        "Coefficient8x2y": c8,
        "Coefficient9xy2": c9,
        "Coefficient10y3": c10,
        "MinimumValueofx": min_x,
        "MaximumValueofx": max_x,
        "MinimumValueofy": min_y,
        "MaximumValueofy": max_y,
        "InputUnitTypeforX": "Temperature",
        "InputUnitTypeforY": "Temperature",
        "OutputUnitType": "Dimensionless"
    }

    return curve_json

#### Load compresor curves

In [214]:
def load_and_print_compressor_curves(db_path, selected_template, verbose=True):
    mt_power_curve = get_compressor_curve(db_path, selected_template, "MT", curve_type="power")
    mt_capacity_curve = get_compressor_curve(db_path, selected_template, "MT", curve_type="capacity")
    lt_power_curve = get_compressor_curve(db_path, selected_template, "LT", curve_type="power")
    lt_capacity_curve = get_compressor_curve(db_path, selected_template, "LT", curve_type="capacity")

    if verbose:
        print("📈 MT Power Curve JSON:")
        print(json.dumps(mt_power_curve, indent=4) if mt_power_curve else "❌ No MT power curve found.")
        print("\n📈 MT Capacity Curve JSON:")
        print(json.dumps(mt_capacity_curve, indent=4) if mt_capacity_curve else "❌ No MT capacity curve found.")
        print("\n📈 LT Power Curve JSON:")
        print(json.dumps(lt_power_curve, indent=4) if lt_power_curve else "❌ No LT power curve found.")
        print("\n📈 LT Capacity Curve JSON:")
        print(json.dumps(lt_capacity_curve, indent=4) if lt_capacity_curve else "❌ No LT capacity curve found.")

    return mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve

#### test

In [215]:
mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve = \
    load_and_print_compressor_curves(db_path, selected_template)

📈 MT Power Curve JSON:
{
    "type": "OS:Curve:Bicubic",
    "name": "advanced_Med_Temp_Comp_Pwr_Curve",
    "Coefficient1Constant": 7438.922294,
    "Coefficient2x": -269.5975647,
    "Coefficient3x2": -9.856104461,
    "Coefficient4y": 553.5732602,
    "Coefficient5y2": -3.385889614,
    "Coefficient6xy": 14.86578782,
    "Coefficient7x3": -0.100796292,
    "Coefficient8x2y": 0.009822793,
    "Coefficient9xy2": 0.097362356,
    "Coefficient10y3": -0.039675217,
    "MinimumValueofx": -23.3,
    "MaximumValueofx": 7.2,
    "MinimumValueofy": 10.0,
    "MaximumValueofy": 60.0,
    "InputUnitTypeforX": "Temperature",
    "InputUnitTypeforY": "Temperature",
    "OutputUnitType": "Dimensionless"
}

📈 MT Capacity Curve JSON:
{
    "type": "OS:Curve:Bicubic",
    "name": "advanced_Med_Temp_Comp_Cap_Curve",
    "Coefficient1Constant": 61412.17466,
    "Coefficient2x": 1570.851906,
    "Coefficient3x2": 13.53344913,
    "Coefficient4y": -500.1952361,
    "Coefficient5y2": 8.16391243,
    "Coef

#### generate compressor objects

In [209]:
def generate_compressor_objects(compressor_info, template, operation_type, curve_json=None):
    """
    Generate RefrigerationCompressor OpenStudio JSON objects including performance curve.

    Args:
        compressor_info (list): List of dicts with rack_number, rack_load, compressors_needed
        template (str): 'old', 'new', or 'advanced'
        operation_type (str): 'MT' or 'LT'
        curve_json (dict): Performance curve JSON (optional)

    Returns:
        List[dict]: List of RefrigerationCompressor JSON objects
    """
    compressor_objects = []

    # Get specs for this template & operation type
    capacity_w, power_w, cop, eer = get_compressor_specs(template, operation_type)

    curve_name = curve_json.get("name") if curve_json else None
        
    for rack in compressor_info:
        rack_number = rack['rack_number']
        actual_needed = rack['compressors_needed']

        # fix to 15 compressors
        model_compressors = max(actual_needed, 15)

        for i in range(1, model_compressors + 1):
            comp_name = f"{template.upper()}_{operation_type}_Rack{rack_number}_Comp{i}"
            comp_obj = {
                "type": "OS:Refrigeration:Compressor",
                "name": comp_name,
                "RatedPowerConsumption": power_w,
                "RatedCapacity": capacity_w,
                "RefrigerantOilCoolerPower": 0,
                "EndUseSubcategory": f"{operation_type}_Compressor_Rack{rack_number}"
            }
            if curve_name:
                comp_obj["CompressorCurve"] = curve_name

            compressor_objects.append(comp_obj)

    return compressor_objects

In [216]:
def prepare_and_store_compressor_objects(mt_info, lt_info, template, db_path):
    # 1. Load performance curves from database
    mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve = load_and_print_compressor_curves(db_path, template, verbose=False)
    # 2. Generate compressor objects using power curves
    mt_compressors = generate_compressor_objects(mt_info, template, "MT", curve_json=mt_power_curve)
    lt_compressors = generate_compressor_objects(lt_info, template, "LT", curve_json=lt_power_curve)

    # 3. Store all in global scope
    globals()["mt_compressors"] = mt_compressors
    globals()["lt_compressors"] = lt_compressors
    globals()["mt_power_curve"] = mt_power_curve
    globals()["mt_capacity_curve"] = mt_capacity_curve
    globals()["lt_power_curve"] = lt_power_curve
    globals()["lt_capacity_curve"] = lt_capacity_curve

    print("✅ Compressor objects and performance curves stored in globals.")


In [217]:
prepare_and_store_compressor_objects(mt_info, lt_info, selected_template, db_path)

✅ Compressor objects and performance curves stored in globals.


#### export to JSON files

In [193]:
def export_existing_condensers_to_json_from_globals(output_path="All_Condensers.json"):
    """
    Automatically generate and export condenser objects + fan power curves 
    using globally defined `mt_info` and `lt_info`.

    Saves the result to JSON and prints a preview to the screen.
    """
    global_vars = globals()
    
    required_vars = ["mt_info", "lt_info"]
    missing = [var for var in required_vars if var not in global_vars]
    if missing:
        raise ValueError(f"❌ Missing required global variables: {missing}")

    # Generate condenser and curve objects
    mt_condensers, mt_curves = generate_condenser_objects(global_vars["mt_info"], "MT")
    lt_condensers, lt_curves = generate_condenser_objects(global_vars["lt_info"], "LT")

    objects = mt_condensers + lt_condensers + mt_curves + lt_curves

    # Add thermal zones
    for zone in ["MainSales", "ActiveStorage"]:
        objects.insert(0, {
            "type": "OS:ThermalZone",
            "name": zone
        })

    openstudio_json = {
        "Version": "0.2.1",
        "Building": get_building_name(),
        "objects": objects
    }

    with open(output_path, "w") as f:
        json.dump(openstudio_json, f, indent=4)

    print(f"✅ Condensers + Curves with zones saved to: {output_path}")
    print("\n📤 Condenser JSON Preview:")
    print(json.dumps(openstudio_json, indent=2))

In [194]:
export_existing_compressors_to_json_from_globals("All_Compressors.json")

✅ Compressor + Curve JSON with zones saved to: All_Compressors.json

📦 OpenStudio JSON Preview:

{
  "Version": "0.2.1",
  "Building": "SuperMarket",
  "objects": [
    {
      "type": "OS:Curve:Bicubic",
      "name": "advanced_Med_Temp_Comp_Pwr_Curve",
      "Coefficient1Constant": 7438.922294,
      "Coefficient2x": -269.5975647,
      "Coefficient3x2": -9.856104461,
      "Coefficient4y": 553.5732602,
      "Coefficient5y2": -3.385889614,
      "Coefficient6xy": 14.86578782,
      "Coefficient7x3": -0.100796292,
      "Coefficient8x2y": 0.009822793,
      "Coefficient9xy2": 0.097362356,
      "Coefficient10y3": -0.039675217,
      "MinimumValueofx": -23.3,
      "MaximumValueofx": 7.2,
      "MinimumValueofy": 10.0,
      "MaximumValueofy": 60.0,
      "InputUnitTypeforX": "Temperature",
      "InputUnitTypeforY": "Temperature",
      "OutputUnitType": "Dimensionless"
    },
    {
      "type": "OS:Curve:Bicubic",
      "name": "advanced_Med_Temp_Comp_Cap_Curve",
      "Coefficient

#### Generate condenser objects

In [205]:
def generate_condenser_objects(rack_info, operation_type):
    """
    Generate OS:Refrigeration:Condenser:AirCooled objects and corresponding performance curves
    for each rack based on the rack load and operation type (MT or LT).

    Args:
        rack_info (list): List of dicts with rack_number and rack_load
        operation_type (str): 'MT' or 'LT'

    Returns:
        Tuple[List[dict], List[dict]]: condensers, curves
    """
    condensers = []
    curves = []

    for rack in rack_info:
        rack_num = rack['rack_number']
        load = rack['rack_load']

        # Condenser capacity
        if operation_type == "LT":
            cond_capacity = 1.2 * load * (1 + 1 / 1.3)
        elif operation_type == "MT":
            cond_capacity = 1.2 * load * (1 + 1 / 2.0)
        else:
            raise ValueError("Invalid operation type. Must be 'MT' or 'LT'.")

        fan_power = 0.0441 * cond_capacity + 695
        condenser_name = f"{operation_type}_Rack{rack_num}_Condenser"
        curve_name = f"{condenser_name}_FanCurve"

        condensers.append({
            "type": "OS:Refrigeration:Condenser:AirCooled",
            "name": condenser_name,
            "RatedEffectiveTotalHeatRejectionRate": cond_capacity,
            "FanPower": fan_power,
            "RatedSubcoolingTemperatureDifference": 5 if operation_type == "MT" else 0,
            "FanPowerCurve": curve_name
        })

        curves.append({
            "type": "OS:Curve:Linear",
            "name": curve_name,
            "Coefficient1Constant": 0,
            "Coefficient2x": cond_capacity / 5.6,
            "MinimumValueofx": 0,
            "MaximumValueofx": 1,
            "InputUnitTypeforX": "Dimensionless",
            "OutputUnitType": "Dimensionless"
        })

    return condensers, curves
def prepare_and_store_condenser_objects(mt_info, lt_info):
    mt_condensers, mt_curves = generate_condenser_objects(mt_info, "MT")
    lt_condensers, lt_curves = generate_condenser_objects(lt_info, "LT")

    globals()["mt_condensers"] = mt_condensers
    globals()["lt_condensers"] = lt_condensers
    globals()["mt_curves"] = mt_curves
    globals()["lt_curves"] = lt_curves

    print("✅ Condenser and curve objects generated and stored in globals.")

In [206]:
prepare_and_store_condenser_objects(mt_info, lt_info)

✅ Condenser and curve objects generated and stored in globals.


#### Generate JSON files

In [196]:
def export_existing_condensers_to_json_from_globals(output_path="All_Condensers.json"):
    """
    Automatically generate and export condenser objects + fan power curves 
    using globally defined `mt_info` and `lt_info`.

    Saves the result to JSON and prints a preview to the screen.
    """
    global_vars = globals()
    
    required_vars = ["mt_info", "lt_info"]
    missing = [var for var in required_vars if var not in global_vars]
    if missing:
        raise ValueError(f"❌ Missing required global variables: {missing}")


    objects = mt_condensers + lt_condensers + mt_curves + lt_curves

    # Add thermal zones at the beginning
    for zone in ["MainSales", "ActiveStorage"]:
        objects.insert(0, {
            "type": "OS:ThermalZone",
            "name": zone
        })

    openstudio_json = {
        "Version": "0.2.1",
        "Building": get_building_name(),
        "objects": objects
    }

    # Save to file
    with open(output_path, "w") as f:
        json.dump(openstudio_json, f, indent=4)

    print(f"✅ Condensers + Curves with zones saved to: {output_path}")
    print("\n📤 Condenser JSON Preview:")
    print(json.dumps(openstudio_json, indent=2))

#### test

In [197]:
export_existing_condensers_to_json_from_globals("All_Condensers.json")

✅ Condensers + Curves with zones saved to: All_Condensers.json

📤 Condenser JSON Preview:
{
  "Version": "0.2.1",
  "Building": "Automated Example",
  "objects": [
    {
      "type": "OS:ThermalZone",
      "name": "ActiveStorage"
    },
    {
      "type": "OS:ThermalZone",
      "name": "MainSales"
    },
    {
      "type": "OS:Refrigeration:Condenser:AirCooled",
      "name": "MT_Rack1_Condenser",
      "RatedEffectiveTotalHeatRejectionRate": 146960.352,
      "FanPower": 7175.9515232,
      "RatedSubcoolingTemperatureDifference": 5,
      "FanPowerCurve": "MT_Rack1_Condenser_FanCurve"
    },
    {
      "type": "OS:Refrigeration:Condenser:AirCooled",
      "name": "MT_Rack2_Condenser",
      "RatedEffectiveTotalHeatRejectionRate": 45238.17599999999,
      "FanPower": 2690.0035615999996,
      "RatedSubcoolingTemperatureDifference": 5,
      "FanPowerCurve": "MT_Rack2_Condenser_FanCurve"
    },
    {
      "type": "OS:Refrigeration:Condenser:AirCooled",
      "name": "MT_Rack3_Con

#### Generate cases and walkins list

In [198]:
def generate_refrigeration_system_objects_from_globals(
    system_name_prefix="Supermarket Rack",
    compressor_prefix="Compressor_List",
    condenser_prefix="Condenser",
    refrigerant="R404A",
    suction_temp=-6.7,
    min_condensing_temp=21.1,
    end_use_category="Refrigeration"
):
    """
    Generate RefrigerationSystem and RefrigerationCaseAndWalkInList objects from global variables.
    Assumes global variables: selected_case_units, selected_walkin_units, mt_racks, lt_racks
    Returns a list of system and case-list JSON objects.
    """
    from itertools import count

    global_vars = globals()

    selected_case_units = global_vars.get("selected_case_units", [])
    selected_walkin_units = global_vars.get("selected_walkin_units", [])
    mt_racks = global_vars.get("mt_racks", [])
    lt_racks = global_vars.get("lt_racks", [])

    if not selected_case_units and not selected_walkin_units:
        raise ValueError("Selected case or walk-in units not found.")

    all_units = selected_case_units + selected_walkin_units
    name_map = {u.case_name: u.osm_name for u in selected_case_units}
    name_map.update({u.walkin_name: u.osm_name for u in selected_walkin_units})

    system_objects = []

    rack_id_gen = count(1)

    def create_objects_for_rack(rack, rack_type):
        rack_number = next(rack_id_gen)
        case_and_walkin_names = [name_map.get(item["name"], item["name"]) for item in rack]

        system_name = f"{system_name_prefix} {rack_type} {rack_number}"
        compressor_list_name = f"{compressor_prefix}_{rack_type}_Rack{rack_number}"
        condenser_name = f"{condenser_prefix}_{rack_type}_Rack{rack_number}"
        list_name = f"{system_name}_CaseWalkinList"

        system = {
            "type": "OS:Refrigeration:System",
            "name": system_name,
            "CompressorListName": compressor_list_name,
            "CondenserName": condenser_name,
            "CaseAndWalkInListName": list_name,
            "RefrigerantType": refrigerant,
            "SuctionTemperature": suction_temp,
            "MinimumCondensingTemperature": min_condensing_temp,
            "EndUseSubcategory": end_use_category
        }

        case_list = {
            "type": "OS:Refrigeration:CaseAndWalkInList",
            "name": list_name,
            "CaseAndWalkInNames": case_and_walkin_names
        }

        return system, case_list

    for rack in mt_racks:
        system_obj, case_list_obj = create_objects_for_rack(rack, "MT")
        system_objects.extend([system_obj, case_list_obj])

    for rack in lt_racks:
        system_obj, case_list_obj = create_objects_for_rack(rack, "LT")
        system_objects.extend([system_obj, case_list_obj])

    return system_objects

In [228]:
def prepare_and_store_refrigeration_system_objects():
    """
    Generate refrigeration system and case/walk-in list objects
    and store them into global variable `refrigeration_system_objects`.
    Requires global variables: mt_racks, lt_racks, selected_case_units, selected_walkin_units
    """
    global_vars = globals()

    required_vars = ["mt_racks", "lt_racks", "selected_case_units", "selected_walkin_units"]
    missing = [var for var in required_vars if var not in global_vars]
    if missing:
        raise ValueError(f"❌ Missing required global variables: {missing}")

    system_objects = generate_refrigeration_system_objects_from_globals()
    globals()["refrigeration_system_objects"] = system_objects

    print("✅ Refrigeration system objects stored in globals.")

In [229]:
prepare_and_store_refrigeration_system_objects()

✅ Refrigeration system objects stored in globals.


### export full system to JSON

In [235]:
def export_full_refrigeration_system_to_json_from_globals(
    output_path="Full_Refrigeration_System.json",
    building_name=None
):
    """
    Export a full OpenStudio refrigeration system JSON using globally defined objects.

    Requires the following global variables to be defined:
    - Zones: implicitly added as MainSales and ActiveStorage
    - Compressor data: mt_compressors, lt_compressors, mt_power_curve, mt_capacity_curve, lt_power_curve, lt_capacity_curve
    - Condenser data: mt_condensers, lt_condensers, mt_curves, lt_curves
    - Refrigeration equipment: case_objects, walkin_objects, refrigeration_system_objects

    Args:
        output_path (str): File path to save the JSON.
        building_name (str): Optional. If None, determines based on global `mode` variable.
        max_preview_chars (int): Number of characters to print as preview.
    """
    global_vars = globals()

    # Define zones
    zones = [
        {"type": "OS:ThermalZone", "name": "MainSales"},
        {"type": "OS:ThermalZone", "name": "ActiveStorage"}
    ]

    required_vars = [
        "mt_compressors", "lt_compressors",
        "mt_power_curve", "mt_capacity_curve",
        "lt_power_curve", "lt_capacity_curve",
        "mt_condensers", "lt_condensers",
        "mt_curves", "lt_curves",
        "case_objects", "walkin_objects",
        "refrigeration_system_objects"
    ]
    missing = [var for var in required_vars if var not in global_vars]
    if missing:
        raise ValueError(f"❌ Missing global variables: {missing}")

    # Determine building name
    if building_name is None:
        mode = global_vars.get("mode", "user").lower()
        building_name = "SuperMarket" if mode == "automated" else "User Defined Example"

    # Collect all objects
    all_objects = (
        zones +
        [global_vars["mt_power_curve"], global_vars["mt_capacity_curve"],
         global_vars["lt_power_curve"], global_vars["lt_capacity_curve"]] +
        global_vars["mt_compressors"] + global_vars["lt_compressors"] +
        global_vars["mt_condensers"] + global_vars["lt_condensers"] +
        global_vars["mt_curves"] + global_vars["lt_curves"] +
        global_vars["case_objects"] + global_vars["walkin_objects"] +
        global_vars["refrigeration_system_objects"]
    )

    # Assemble final JSON
    openstudio_json = {
        "Version": "0.2.1",
        "Building": building_name,
        "objects": all_objects
    }

    # Save to file
    with open(output_path, "w") as f:
        json.dump(openstudio_json, f, indent=2)

    print(f"✅ Full OpenStudio Refrigeration JSON saved to: {output_path}")
    print("\n📦 Preview:")
    print(json.dumps(openstudio_json, indent=2))

In [236]:
export_full_refrigeration_system_to_json_from_globals()

✅ Full OpenStudio Refrigeration JSON saved to: Full_Refrigeration_System.json

📦 Preview:
{
  "Version": "0.2.1",
  "Building": "SuperMarket",
  "objects": [
    {
      "type": "OS:ThermalZone",
      "name": "MainSales"
    },
    {
      "type": "OS:ThermalZone",
      "name": "ActiveStorage"
    },
    {
      "type": "OS:Curve:Bicubic",
      "name": "advanced_Med_Temp_Comp_Pwr_Curve",
      "Coefficient1Constant": 7438.922294,
      "Coefficient2x": -269.5975647,
      "Coefficient3x2": -9.856104461,
      "Coefficient4y": 553.5732602,
      "Coefficient5y2": -3.385889614,
      "Coefficient6xy": 14.86578782,
      "Coefficient7x3": -0.100796292,
      "Coefficient8x2y": 0.009822793,
      "Coefficient9xy2": 0.097362356,
      "Coefficient10y3": -0.039675217,
      "MinimumValueofx": -23.3,
      "MaximumValueofx": 7.2,
      "MinimumValueofy": 10.0,
      "MaximumValueofy": 60.0,
      "InputUnitTypeforX": "Temperature",
      "InputUnitTypeforY": "Temperature",
      "OutputUni